In [6]:
import numpy
import matplotlib.pyplot as plt
import pandas
import math
import random
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Merge
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

def create_dataset(dataset, look_back=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-look_back-1):
        a = dataset[i:(i+look_back)]
        dataX.append(a)
        dataY.append(dataset[i + look_back])
    return numpy.array(dataX), numpy.array(dataY)

corMat = pandas.read_csv('correlationmatrix.csv', engine='python')
corMat = corMat.iloc[:,1:21]

dataframe = pandas.read_csv('시뮬레이션 데이터.csv', engine='python')
dataframe = dataframe.iloc[0:46,1:21]
error_scores2= pandas.DataFrame(index=range(20),columns=["5~10년","5~15년","5~20년","5~25년","5~30년","5~35년"])

for i in range(20):
	k=0
	dataframe2 = dataframe.iloc[:,i]
	dataset = dataframe2.values
	dataset = dataset.astype('float32')
	scaler = MinMaxScaler(feature_range=(0, 1))
	dataset = scaler.fit_transform(dataset)
	
	datasetcompare1 = (dataframe.iloc[:,corMat.iloc[i,1]]).values
	datasetcompare1 = datasetcompare1.astype('float32')
	scalercompare1 = MinMaxScaler(feature_range=(0, 1))
	datasetcompare1 = scalercompare1.fit_transform(datasetcompare1)
	

	datasetcompare2 = (dataframe.iloc[:,corMat.iloc[i,2]]).values
	datasetcompare2 = datasetcompare2.astype('float32')
	scalercompare2 = MinMaxScaler(feature_range=(0, 1))
	datasetcompare2 = scalercompare2.fit_transform(datasetcompare2)

	repeats = 3
	
	for j in [5,10,15,20,25,30]:
		error_scores = list()
		for r in range(repeats):
			numpy.random.seed(random.randrange(1,1000))
			train_size = j
			test_size = len(dataset) - train_size
			train, test = dataset[0:train_size], dataset[train_size:len(dataset)]
			look_back = 1
			trainX, trainY = create_dataset(train, look_back)
			testX, testY = create_dataset(test, look_back)
			trainX = numpy.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
			testX = numpy.reshape(testX, (testX.shape[0], 1, testX.shape[1]))
			batch_size = 1
			first_model = Sequential()
			first_model.add(LSTM(4, batch_input_shape=(batch_size, look_back, 1), stateful=True, return_sequences=True))
			first_model.add(LSTM(4, batch_input_shape=(batch_size, look_back, 1), stateful=True))
			first_model.add(Dense(1))
			
			
			traincompare1, testcompare1 = datasetcompare1[0:train_size], datasetcompare1[train_size:len(dataset)]
			look_back = 1
			trainXcompare1, trainYcompare1 = create_dataset(traincompare1, look_back)
			testXcompare1, testYcompare1 = create_dataset(testcompare1, look_back)
			trainXcompare1 = numpy.reshape(trainXcompare1, (trainXcompare1.shape[0], 1, trainXcompare1.shape[1]))
			testXcompare1 = numpy.reshape(testXcompare1, (testXcompare1.shape[0], 1, testXcompare1.shape[1]))
			
			second_model = Sequential()
			second_model.add(LSTM(4, batch_input_shape=(batch_size, look_back, 1), stateful=True, return_sequences=True))
			second_model.add(LSTM(4, batch_input_shape=(batch_size, look_back, 1), stateful=True))
			second_model.add(Dense(1))
			
			
			traincompare2, testcompare2 = datasetcompare2[0:train_size], datasetcompare2[train_size:len(dataset)]
			look_back = 1
			trainXcompare2, trainYcompare2 = create_dataset(traincompare2, look_back)
			testXcompare2, testYcompare2 = create_dataset(testcompare2, look_back)
			trainXcompare2 = numpy.reshape(trainXcompare2, (trainXcompare2.shape[0], 1, trainXcompare2.shape[1]))
			testXcompare2 = numpy.reshape(testXcompare2, (testXcompare2.shape[0], 1, testXcompare2.shape[1]))
			
			third_model = Sequential()
			third_model.add(LSTM(4, batch_input_shape=(batch_size, look_back, 1), stateful=True, return_sequences=True))
			third_model.add(LSTM(4, batch_input_shape=(batch_size, look_back, 1), stateful=True))
			third_model.add(Dense(1))

			
			
			
			
			
			model = Sequential()
			model.add(Merge([first_model, second_model,third_model], mode='concat'))
			model.add(Dense(1))

			model.compile(loss='mean_squared_error', optimizer='adam')
			for z in range(10):            
				model.fit([trainX,trainXcompare1,trainXcompare2], trainY, epochs=1, batch_size=batch_size, verbose=2, shuffle=False)
				model.reset_states()
			trainPredict = model.predict([trainX,trainXcompare1,trainXcompare2], batch_size=batch_size)
			model.reset_states()
			testPredict = model.predict([testX,testXcompare1,testXcompare2], batch_size=batch_size)
			trainPredict = scaler.inverse_transform(trainPredict)
			trainY = scaler.inverse_transform([trainY])
			testPredict = scaler.inverse_transform(testPredict)
			testY = scaler.inverse_transform([testY])
			
			
			testScore = math.sqrt(mean_squared_error(testY[0], testPredict[:,0]))
			error_scores.append(testScore)
		
		results = pandas.DataFrame()
		results['rmse'] = error_scores
		error_scores2.iloc[i,k]=results.describe().iloc[1,0]
		k=k+1


/home/kang/anaconda2/lib/python2.7/site-packages/sklearn/preprocessing/data.py:324: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
/home/kang/anaconda2/lib/python2.7/site-packages/sklearn/preprocessing/data.py:359: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
/home/kang/anaconda2/lib/python2.7/site-packages/sklearn/preprocessing/data.py:324: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your dat

Epoch 1/1
 - 4s - loss: 0.0151
Epoch 1/1
 - 0s - loss: 0.0120
Epoch 1/1
 - 0s - loss: 0.0085
Epoch 1/1
 - 0s - loss: 0.0054
Epoch 1/1
 - 0s - loss: 0.0033
Epoch 1/1
 - 0s - loss: 0.0020
Epoch 1/1
 - 0s - loss: 0.0015
Epoch 1/1
 - 0s - loss: 0.0017
Epoch 1/1
 - 0s - loss: 0.0020
Epoch 1/1
 - 0s - loss: 0.0024
Epoch 1/1
 - 4s - loss: 0.0150
Epoch 1/1
 - 0s - loss: 0.0120
Epoch 1/1
 - 0s - loss: 0.0088
Epoch 1/1
 - 0s - loss: 0.0061
Epoch 1/1
 - 0s - loss: 0.0040
Epoch 1/1
 - 0s - loss: 0.0026
Epoch 1/1
 - 0s - loss: 0.0018
Epoch 1/1
 - 0s - loss: 0.0016
Epoch 1/1
 - 0s - loss: 0.0017
Epoch 1/1
 - 0s - loss: 0.0019
Epoch 1/1
 - 4s - loss: 0.0147
Epoch 1/1
 - 0s - loss: 0.0115
Epoch 1/1
 - 0s - loss: 0.0084
Epoch 1/1
 - 0s - loss: 0.0057
Epoch 1/1
 - 0s - loss: 0.0037
Epoch 1/1
 - 0s - loss: 0.0024
Epoch 1/1
 - 0s - loss: 0.0017
Epoch 1/1
 - 0s - loss: 0.0015
Epoch 1/1
 - 0s - loss: 0.0017
Epoch 1/1
 - 0s - loss: 0.0021
Epoch 1/1
 - 5s - loss: 0.0515
Epoch 1/1
 - 0s - loss: 0.0355
Epoch 1/

/home/kang/anaconda2/lib/python2.7/site-packages/sklearn/preprocessing/data.py:324: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
/home/kang/anaconda2/lib/python2.7/site-packages/sklearn/preprocessing/data.py:359: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
/home/kang/anaconda2/lib/python2.7/site-packages/sklearn/preprocessing/data.py:324: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your dat

Epoch 1/1
 - 9s - loss: 0.0166
Epoch 1/1
 - 0s - loss: 0.0132
Epoch 1/1
 - 0s - loss: 0.0102
Epoch 1/1
 - 0s - loss: 0.0078
Epoch 1/1
 - 0s - loss: 0.0057
Epoch 1/1
 - 0s - loss: 0.0042
Epoch 1/1
 - 0s - loss: 0.0030
Epoch 1/1
 - 0s - loss: 0.0023
Epoch 1/1
 - 0s - loss: 0.0020
Epoch 1/1
 - 0s - loss: 0.0019
Epoch 1/1
 - 11s - loss: 0.0164
Epoch 1/1
 - 0s - loss: 0.0124
Epoch 1/1
 - 0s - loss: 0.0087
Epoch 1/1
 - 0s - loss: 0.0056
Epoch 1/1
 - 0s - loss: 0.0034
Epoch 1/1
 - 0s - loss: 0.0021
Epoch 1/1
 - 0s - loss: 0.0017
Epoch 1/1
 - 0s - loss: 0.0018
Epoch 1/1
 - 0s - loss: 0.0023
Epoch 1/1
 - 0s - loss: 0.0028
Epoch 1/1
 - 11s - loss: 0.0160
Epoch 1/1
 - 0s - loss: 0.0109
Epoch 1/1
 - 0s - loss: 0.0066
Epoch 1/1
 - 0s - loss: 0.0037
Epoch 1/1
 - 0s - loss: 0.0021
Epoch 1/1
 - 0s - loss: 0.0016
Epoch 1/1
 - 0s - loss: 0.0019
Epoch 1/1
 - 0s - loss: 0.0025
Epoch 1/1
 - 0s - loss: 0.0030
Epoch 1/1
 - 0s - loss: 0.0032
Epoch 1/1
 - 11s - loss: 0.0401
Epoch 1/1
 - 0s - loss: 0.0172
Epoch

/home/kang/anaconda2/lib/python2.7/site-packages/sklearn/preprocessing/data.py:324: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
/home/kang/anaconda2/lib/python2.7/site-packages/sklearn/preprocessing/data.py:359: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
/home/kang/anaconda2/lib/python2.7/site-packages/sklearn/preprocessing/data.py:324: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your dat

...

Epoch 1/1
 - 112s - loss: 0.0155
Epoch 1/1
 - 0s - loss: 0.0097
Epoch 1/1
 - 0s - loss: 0.0047
Epoch 1/1
 - 0s - loss: 0.0018
Epoch 1/1
 - 0s - loss: 0.0015
Epoch 1/1
 - 0s - loss: 0.0031
Epoch 1/1
 - 0s - loss: 0.0052
Epoch 1/1
 - 0s - loss: 0.0061
Epoch 1/1
 - 0s - loss: 0.0056
Epoch 1/1
 - 0s - loss: 0.0043
Epoch 1/1
 - 113s - loss: 0.0162
Epoch 1/1
 - 0s - loss: 0.0117
Epoch 1/1
 - 0s - loss: 0.0076
Epoch 1/1
 - 0s - loss: 0.0044
Epoch 1/1
 - 0s - loss: 0.0024
Epoch 1/1
 - 0s - loss: 0.0016
Epoch 1/1
 - 0s - loss: 0.0018
Epoch 1/1
 - 0s - loss: 0.0025
Epoch 1/1
 - 0s - loss: 0.0032
Epoch 1/1
 - 0s - loss: 0.0036
Epoch 1/1
 - 112s - loss: 0.0163
Epoch 1/1
 - 0s - loss: 0.0118
Epoch 1/1
 - 0s - loss: 0.0076
Epoch 1/1
 - 0s - loss: 0.0045
Epoch 1/1
 - 0s - loss: 0.0026
Epoch 1/1
 - 0s - loss: 0.0017
Epoch 1/1
 - 0s - loss: 0.0017
Epoch 1/1
 - 0s - loss: 0.0022
Epoch 1/1
 - 0s - loss: 0.0027
Epoch 1/1
 - 0s - loss: 0.0030
Epoch 1/1
 - 108s - loss: 0.0543
Epoch 1/1
 - 0s - loss: 0.0287


/home/kang/anaconda2/lib/python2.7/site-packages/sklearn/preprocessing/data.py:324: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
/home/kang/anaconda2/lib/python2.7/site-packages/sklearn/preprocessing/data.py:359: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
/home/kang/anaconda2/lib/python2.7/site-packages/sklearn/preprocessing/data.py:324: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your dat

Epoch 1/1
 - 107s - loss: 0.0150
Epoch 1/1
 - 0s - loss: 0.0105
Epoch 1/1
 - 0s - loss: 0.0064
Epoch 1/1
 - 0s - loss: 0.0035
Epoch 1/1
 - 0s - loss: 0.0019
Epoch 1/1
 - 0s - loss: 0.0015
Epoch 1/1
 - 0s - loss: 0.0019
Epoch 1/1
 - 0s - loss: 0.0025
Epoch 1/1
 - 0s - loss: 0.0030
Epoch 1/1
 - 0s - loss: 0.0031
Epoch 1/1
 - 122s - loss: 0.0160
Epoch 1/1
 - 0s - loss: 0.0135
Epoch 1/1
 - 0s - loss: 0.0107
Epoch 1/1
 - 0s - loss: 0.0081
Epoch 1/1
 - 0s - loss: 0.0060
Epoch 1/1
 - 0s - loss: 0.0044
Epoch 1/1
 - 0s - loss: 0.0032
Epoch 1/1
 - 0s - loss: 0.0024
Epoch 1/1
 - 0s - loss: 0.0020
Epoch 1/1
 - 0s - loss: 0.0018
Epoch 1/1
 - 115s - loss: 0.0163
Epoch 1/1
 - 0s - loss: 0.0143
Epoch 1/1
 - 0s - loss: 0.0114
Epoch 1/1
 - 0s - loss: 0.0084
Epoch 1/1
 - 0s - loss: 0.0058
Epoch 1/1
 - 0s - loss: 0.0038
Epoch 1/1
 - 0s - loss: 0.0024
Epoch 1/1
 - 0s - loss: 0.0018
Epoch 1/1
 - 0s - loss: 0.0017
Epoch 1/1
 - 0s - loss: 0.0019
Epoch 1/1
 - 113s - loss: 0.0509
Epoch 1/1
 - 0s - loss: 0.0222


/home/kang/anaconda2/lib/python2.7/site-packages/sklearn/preprocessing/data.py:324: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
/home/kang/anaconda2/lib/python2.7/site-packages/sklearn/preprocessing/data.py:359: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
/home/kang/anaconda2/lib/python2.7/site-packages/sklearn/preprocessing/data.py:324: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your dat

Epoch 1/1
 - 129s - loss: 0.0185
Epoch 1/1
 - 0s - loss: 0.0172
Epoch 1/1
 - 0s - loss: 0.0134
Epoch 1/1
 - 0s - loss: 0.0086
Epoch 1/1
 - 0s - loss: 0.0045
Epoch 1/1
 - 0s - loss: 0.0021
Epoch 1/1
 - 0s - loss: 0.0016
Epoch 1/1
 - 0s - loss: 0.0024
Epoch 1/1
 - 0s - loss: 0.0037
Epoch 1/1
 - 0s - loss: 0.0046
Epoch 1/1
 - 124s - loss: 0.0180
Epoch 1/1
 - 0s - loss: 0.0156
Epoch 1/1
 - 0s - loss: 0.0120
Epoch 1/1
 - 0s - loss: 0.0080
Epoch 1/1
 - 0s - loss: 0.0045
Epoch 1/1
 - 0s - loss: 0.0023
Epoch 1/1
 - 0s - loss: 0.0016
Epoch 1/1
 - 0s - loss: 0.0022
Epoch 1/1
 - 0s - loss: 0.0034
Epoch 1/1
 - 0s - loss: 0.0044
Epoch 1/1
 - 126s - loss: 0.0179
Epoch 1/1
 - 0s - loss: 0.0159
Epoch 1/1
 - 0s - loss: 0.0134
Epoch 1/1
 - 0s - loss: 0.0107
Epoch 1/1
 - 0s - loss: 0.0080
Epoch 1/1
 - 0s - loss: 0.0057
Epoch 1/1
 - 0s - loss: 0.0039
Epoch 1/1
 - 0s - loss: 0.0027
Epoch 1/1
 - 0s - loss: 0.0020
Epoch 1/1
 - 0s - loss: 0.0018
Epoch 1/1
 - 129s - loss: 0.0838
Epoch 1/1
 - 0s - loss: 0.0778


In [7]:
pandas.DataFrame((error_scores2.mean(axis=0))).iloc[:,0]

5~10년    7.824383
5~15년    6.806560
5~20년    5.077948
5~25년    3.746393
5~30년    3.300193
5~35년    2.415593
Name: 0, dtype: float64

In [5]:
error_scores2

,5~10년,5~15년,5~20년,5~25년,5~30년,5~35년
0,1.83603,1.86803,1.07727,1.24306,0.677789,0.57066
1,10.7766,4.90068,4.88338,4.47551,2.60515,2.80434
2,25.239,20.2125,11.8599,9.50423,6.82142,6.26115
3,6.43055,3.89667,3.35897,1.76797,1.44202,1.4396
4,1.13379,1.23307,0.663647,0.56606,0.428662,0.463078
5,17.5158,13.2341,8.02066,4.51952,4.74361,4.31852
6,6.02164,6.03738,2.53474,2.99439,2.53254,2.09073
7,1.84939,1.24998,0.886448,0.666886,0.680339,0.39187
8,0.949075,0.458436,0.441384,0.440924,0.288646,0.312124
9,8.52761,7.30641,5.52802,3.10286,2.75724,1.67397
